In [1]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimlab.data_profile.' + cim_profile)
sparql = importlib.import_module('cimlab.loaders.sparql.' + cim_profile)

In [2]:
from cimlab.loaders import Parameter, ConnectionParameters
from cimlab.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimlab.loaders.gridappsd import GridappsdConnection, get_topology_response
from cimlab.models import DistributedModel
# import cim.data_profile as cim
import json


In [3]:
# # Blazegraph connection for running inside the container
# params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
# bg = BlazegraphConnection(params)

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')


In [5]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [6]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [7]:
# from gridappsd import GridAPPSD
from topology_processor import DistributedTopology

gapps = GridappsdConnection(feeder_mrid)
Topology = DistributedTopology(gapps, feeder_mrid)
topology_response = Topology.create_switch_areas(feeder_mrid)
# topology_response = get_topology_response(feeder_mrid)
topology_response = json.loads(topology_response)
# with open('topo_message.json') as topo_file:
#     topology_response = json.load(topo_file)


/home/d3m614/.local/lib/python3.8/site-packages/ipykernel_launcher.py


In [8]:
# topology_response = json.loads(topology_response)

In [9]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response['feeders'])

In [10]:
switch_area = network.switch_areas[0]

# Get phase, bus info about ACLineSegments
switch_area.get_all_attributes(cim.ACLineSegment)
switch_area.get_all_attributes(cim.ACLineSegmentPhase)
switch_area.get_all_attributes(cim.Terminal)

line_ids = list(switch_area.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', switch_area.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in switch_area.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
    

line mrid:  _0BBD0EA3-F665-465B-86FD-FC8B8466AD53
line name: 645646
bus 1:  _8C58660F-C62C-4903-BE72-22F1255B1E62
bus 2:  _0124E881-B82D-4206-BBDF-37D585159872
phase  ['C'] :  _94FCAFFF-E5F7-4F97-B697-35C46656EDCA
phase  ['B'] :  _CE2AFB05-A1B1-4082-B4EB-2AD8A1D3422D
line mrid:  _1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0
line name: 632645
bus 1:  _0124E881-B82D-4206-BBDF-37D585159872
bus 2:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  ['C'] :  _19B6044B-6915-4245-9DD1-6AB0444D42A6
phase  ['B'] :  _D9C9CEA6-B3F0-4E02-9911-3DFA192B4D01
line mrid:  _A04CDFB1-E951-4FC4-8882-0323CD70AE3C
line name: 650632
bus 1:  _673E896A-DCBF-4E43-9924-BEB31C5B6005
bus 2:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  ['A'] :  _2A5A1025-AD81-4BFA-A7F0-ED70F7F43DAA
phase  ['B'] :  _2D60B241-DE87-48BD-A8D3-FE4B705B40ED
phase  ['C'] :  _A66508A9-C9C1-4DB5-8FFD-F081E5E9E14D
line mrid:  _FBE26B35-13AB-457D-9795-DF58B28E309D
line name: 632633
bus 1:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
bus 2:  _C6256170-E6ED-4F91-

In [11]:

switch_area.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases[1].Location.name

'632633_Loc'

In [12]:
switch_area = network.switch_areas[0]

switch_area.get_all_attributes(cim.EnergyConsumer)
switch_area.get_all_attributes(cim.EnergyConsumerPhase)

load_ids = list(switch_area.typed_catalog[cim.EnergyConsumer].keys())

for load in load_ids:
    print('p = ', switch_area.typed_catalog[cim.EnergyConsumer][load].p)
    print('q = ', switch_area.typed_catalog[cim.EnergyConsumer][load].q)
    for load_phs in switch_area.typed_catalog[cim.EnergyConsumer][load].EnergyConsumerPhase:
        print('phases: ', load_phs.phase)
        
json.loads(switch_area.__dumps__(cim.EnergyConsumer))

p =  230000
q =  132000
phases:  ['B']
p =  170000
q =  125000
phases:  ['B']


{'_0BC9183A-9067-4E06-AA5B-48DC2AB30C80': {'mRID': '_0BC9183A-9067-4E06-AA5B-48DC2AB30C80',
  'aliasName': '',
  'description': '',
  'name': '646',
  'Names': [],
  'AssetDatasheet': '',
  'Assets': [],
  'ConfigurationEvent': [],
  'Controls': [],
  'Location': '_9E735173-F2E5-408C-8279-54ED0D3BBDD1',
  'Measurements': ['_7a436400-3a77-4eaf-bb04-ef7954fcbdd9',
   '_e1b4fd36-01d0-4d7d-9223-2279911fe8c3'],
  'OperatingShare': [],
  'PSRType': '',
  'ReportingGroup': [],
  'aggregate': '',
  'inService': '',
  'networkAnalysisEnabled': '',
  'normallyInService': '',
  'AdditionalEquipmentContainer': [],
  'EquipmentContainer': '',
  'Faults': [],
  'OperationalLimitSet': [],
  'UsagePoints': '',
  'BaseVoltage': '_2A158E0C-CD01-4A50-AEBA-59D761FCF15D',
  'SvStatus': [],
  'Terminals': ['_579A122E-A262-4830-88C6-1ADDB81761B1'],
  'customerCount': '1',
  'grounded': 'false',
  'p': '230000',
  'phaseConnection': ['D'],
  'q': '132000',
  'EnergyConsumerPhase': ['_C163738E-2A54-4886-9741-7

In [13]:
network.get_all_attributes(cim.TransformerTank)
network.get_all_attributes(cim.TransformerTankEnd)
network.get_all_attributes(cim.TransformerTankInfo)
network.get_all_attributes(cim.TransformerEndInfo)
network.get_all_attributes(cim.ShortCircuitTest)
network.get_all_attributes(cim.NoLoadTest)
network.get_all_attributes(cim.Terminal)

In [14]:
network_area=switch_area
network_area.get_all_attributes(cim.ACLineSegment)
network_area.get_all_attributes(cim.ACLineSegmentPhase)
network_area.get_all_attributes(cim.PerLengthPhaseImpedance)
network_area.get_all_attributes(cim.PhaseImpedanceData)

network_area.get_all_attributes(cim.WireSpacingInfo)
network_area.get_all_attributes(cim.WirePosition)
network_area.get_all_attributes(cim.OverheadWireInfo)
network_area.get_all_attributes(cim.ConcentricNeutralCableInfo)
network_area.get_all_attributes(cim.TapeShieldCableInfo)

network_area.get_all_attributes(cim.Terminal) #bus info

In [15]:
line_ids = list(network.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', network.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', network.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', network.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in network.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
        if line_phs.WireInfo is not None:
            print('gmr: ', line_phs.WireInfo.gmr)
    if network.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance is not None:
        for impedance in network.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance.PhaseImpedanceData:
            pr int('row:', impedance.row, 'col:', impedance.column, 'r:', impedance.r, 'x:', impedance.x, 'b:', impedance.b)
#     if network.typed_catalog[cim.ACLineSegment][line_id].WireSpacingInfo is not None:
#         for wire_info in network.typed_catalog[cim.ACLineSegment][line_id].WireSpacingInfo.WireAssemblyInfo:
#             print('row:', wire_info.)

SyntaxError: invalid syntax (3208409178.py, line 13)

In [ ]:
print(switch_area.get_attributes_query(cim.ConcentricNeutralCableInfo))

In [ ]:
network.typed_catalog[cim.ACLineSegmentPhase][line_id].AssetInfo

In [ ]:
for tank in network.typed_catalog[cim.TransformerTank].values():
    print('\n name:', tank.name)
    for end in tank.TransformerTankEnds:
        print('end number:', end.endNumber)
        node = end.Terminal.ConnectivityNode
        print('bus: ', node.name, node.mRID)
    
    for end_info in tank.TransformerTankInfo.TransformerEndInfos:
        
        print('end number', end_info.endNumber)
        print('rated voltage:', end_info.ratedU)
        print('resistance:', end_info.r)
        for no_load_test in end_info.EnergisedEndNoLoadTests:
            print('exciting current:', no_load_test.excitingCurrent)

        for short_circuit_test in end_info.EnergisedEndShortCircuitTests:
            print('energisedEndStep:', short_circuit_test.energisedEndStep)
            print('groundedEndStep:', short_circuit_test.groundedEndStep)
            print('leakageImpedance:', short_circuit_test.leakageImpedance)
            
        for short_circuit_test in end_info.GroundedEndShortCircuitTests:
            print('energisedEndStep:', short_circuit_test.energisedEndStep)
            print('groundedEndStep:', short_circuit_test.groundedEndStep)
            print('leakageImpedance:', short_circuit_test.leakageImpedance)


In [ ]:
end = network.typed_catalog[cim.TransformerTank]['_17A934C7-1510-481F-BAD7-189058957FF1'].TransformerTankEnds[0]
end.Terminal
print()

In [ ]:
json.loads(network.__dumps__(cim.TransformerEndInfo))

In [ ]:
print(network.get_attributes_query(cim.TransformerTankEnd))

In [ ]:
secondary_area = network.switch_areas[3].secondary_areas[0]
secondary_area.get_all_attributes(cim.PowerElectronicsConnection)
secondary_area.get_all_attributes(cim.PowerElectronicsConnectionPhase)

pec_ids = list(secondary_area.typed_catalog[cim.PowerElectronicsConnection])

for pec in pec_ids:
    print('p = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].p)
    print('q = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].q)
    for pec_phs in secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].PowerElectronicsConnectionPhases:
        print('phase ', pec_phs.phase, ': ', pec_phs.mRID)

In [ ]:
network_area.typed_catalog[cim.ACLineSegment][line_id]

In [ ]:
import sys
path = sys.argv[0]
if path != '/gridappsd/services/gridappsd-topology-processor':
    print(8890)

In [ ]:
values = ['_0649C1EF-E39F-4B46-AD80-D3C180E4562E']
message = bg.sparql.get_class_type_sparql(feeder_mrid, values)
output = bg.execute(message)
out = output['results']['bindings']
if values and not out:
    print('yes')

In [ ]:
list(network.typed_catalog[cim.ACLineSegment].keys())

In [ ]:
cim.PowerTransformerEnd()

In [ ]:
sparql_message = eval(f"self.sparql.{cim_class.__name__}SPARQL.get_all_attributes('{feeder_mrid}', '{typed_catalog}')")

In [ ]:
sparql_func = getattr(sparql, f"{cim_class.__name__}SPARQL")
sparql_message = sparql_func.get_all_attributes(feeder_mrid, network.typed_catalog)
print(sparql_message)

In [ ]:
json.loads(network.__dumps__(cim.PowerTransformerEnd))

In [ ]:
import re
cim_class = cim.ACLineSegmentPhase
attribute = 'phase'
attribute_type = cim_class.__dataclass_fields__[attribute].type
# if attribute_type in cim.__all__:
at_cls = re.match(r'Optional\[(.*)]',attribute_type)
attribute_class = at_cls.group(1)
if attribute_class in cim.__all__:
    print(attribute_class)

In [ ]:
class_type = eval(f"cim.{attribute_class}")
if type(class_type) is type:
    print('yes')


In [ ]:
cim_class = cim.PowerTransformerEnd
attribute = 'CoreAdmittance'
attribute_type = cim_class.__dataclass_fields__[attribute].type
# if attribute_type in cim.__a
print(attribute_type)

In [ ]:
print(network.get_attributes_query(cim.TransformerCoreAdmittance))

In [ ]:
response['results']['bindings']

In [16]:
network.get_all_attributes(cim.PowerTransformer)
network.get_all_attributes(cim.PowerTransformerInfo)
network.get_all_attributes(cim.PowerTransformerEnd)
network.get_all_attributes(cim.TransformerMeshImpedance)
network.get_all_attributes(cim.TransformerCoreAdmittance)
network.get_all_attributes(cim.Terminal)

for xfmr in network.typed_catalog[cim.PowerTransformer].values():
    print('\n name: ', xfmr.name, xfmr.mRID)
    for end in xfmr.PowerTransformerEnd:
        print('end number:', end.endNumber)
        print('bus:', end.Terminal.ConnectivityNode.name)
        print('connection:', end.connectionKind)
        for mesh_imp in end.ToMeshImpedance:
            print('r:', mesh_imp.r)
            print('x:', mesh_imp.x)
        if end.CoreAdmittance is not None:
            print('g:', end.CoreAdmittance.g)
            print('b:', end.CoreAdmittance.b)
        


 name:  xfm1 _1E6B5C97-C4E8-4CED-B9A5-6E69F389DA93
end number: 1
bus: xf1
connection: ['Y']
g: 0
b: 0
end number: 2
bus: 634
connection: ['Y']
r: 0.3807232
x: 0.692224

 name:  sub3 _259E820F-B4AF-4E1A-8271-687534EDAECC
end number: 2
bus: 650
connection: ['Yn']
end number: 1
bus: sourcebus
connection: ['D']
g: 0
b: 0
end number: 3
bus: 650z
connection: ['Y']
r: 0.02645
x: 0.66125

 name:  Reg _67B57539-590B-4158-9CBB-9DBA2FE6C1F0

 name:  tpoletop _7FF12CEF-1C4D-4CD2-8FCC-662530A7BA33


In [ ]:
network.typed_catalog[cim.PowerTransformerEnd]['_B0352ED5-349E-4511-9663-EC1131757376'].CoreAdmittance